In [63]:
suppressPackageStartupMessages({
    library(scater)
    library(scran)
    library(tidyverse)
    library(ComplexHeatmap)
    library(logger)
})

#  Read SCE data

In [64]:
sce <- readRDS("../../data/expression/sce/sce_Smartseq2_scHCC-CD45_featureCounts_qc_clustered.rds")


# Read integration sites

In [65]:
intSiteHBV <- readr::read_tsv("../../data/intSites_HBV/arriba_fusions_CD45N_20200901.tsv") %>%
    dplyr::filter(!grepl("group", group)) %>%
    rename(gene1 = `#gene1`) %>% 
    dplyr::mutate(chr1 = sapply(strsplit(breakpoint1, ":"), `[[`, 1),
                chr2 = sapply(strsplit(breakpoint2, ":"), `[[`, 1),
                bp_coord1 = as.numeric(sapply(strsplit(breakpoint1, ":"), `[[`, 2)),
                bp_coord2 = as.numeric(sapply(strsplit(breakpoint2, ":"), `[[`, 2))) %>%
    dplyr::mutate(split_reads1 = as.numeric(split_reads1),
                split_reads2 = as.numeric(split_reads2)) %>%
    dplyr::mutate(spanning_counts = split_reads1 + split_reads2) %>% 
    dplyr::mutate(chrHuman = ifelse(grepl("HBV", chr1), chr2, chr1 ),
                coordHuman = ifelse(grepl("HBV", chr1), bp_coord2, bp_coord1), 
                chrVir = "HBV",
                coordVir = ifelse(grepl("HBV", chr1), bp_coord1, bp_coord2)) %>%
    dplyr::mutate(siteTypeHuman = ifelse(grepl("HBV", chr1), site2 , site1 )) %>%
    dplyr::mutate(chrHuman = factor(chrHuman, levels = c(1:22, "X"))) %>% 
   #--- Annotating Intergation Sites genes
   dplyr::mutate(geneHuman = ifelse(grepl("HBV", chr1), gene2, gene1)) %>%
   dplyr::mutate(cell.id = gsub("PD10_ZZM_|_arriba_output.tsv", "", group) )
 

Parsed with column specification:
cols(
  .default = col_character()
)

See spec(...) for full column specifications.



# Integrate meta donor

In [66]:
meta <- as.data.frame(colData(sce))
intSiteHBV <- intSiteHBV %>% left_join(meta[, c("cell.id", "donor", "tissue", "Sample", "leiden_global_celltype")])
intSiteHBV <- intSiteHBV[!is.na(intSiteHBV$donor),]

Joining, by = "cell.id"



# Merge Flanking sites

In [67]:
#--- flanking each sites for 500bp
suppressPackageStartupMessages(library(GenomicAlignments))
log_info("Extending sites 100bp")
humanSites <- GRanges(seqnames = intSiteHBV$chrHuman, ranges = IRanges(intSiteHBV$coordHuman, width = 1), spanning_counts = intSiteHBV$spanning_counts)
#shift(humanSites, 5)
gr <- flank(humanSites, 1000, both = T)
regions <- GenomicRanges::reduce(gr)
#--- match site and regions
ov <- findOverlaps(gr, regions)
sites.split <- split(queryHits(ov), subjectHits(ov))
aa <- as.data.frame(regions[subjectHits(ov),])
humanSites$siteClusters <- paste0(aa$seqnames, ":", aa$start, "-", aa$end)

log_info("Merging...")
intSiteHBV$siteClusters <- humanSites$siteClusters
intSiteHBV$siteHuman <- paste0(intSiteHBV$chrHuman, ":", intSiteHBV$coordHuman)

INFO [2020-10-02 21:07:41] Extending sites 100bp
INFO [2020-10-02 21:07:41] Merging...


In [68]:
intSiteHBV$siteHuman

[1] "9:26713423"   "9:26713422"   "9:26711904"   "9:26713418"   "9:26713406"  
  [6] "9:26713414"   "9:26713423"   "9:26713423"   "9:26713423"   "9:26713423"  
 [11] "9:26713398"   "9:26711145"   "12:111792137" "9:26713423"   "9:26713423"  
 [16] "9:26713423"   "9:26713423"   "9:26713423"   "9:26713423"   "9:26713422"  
 [21] "9:26713423"   "9:26713423"   "9:26713423"   "9:26713420"   "9:26713423"  
 [26] "9:26713423"   "9:26713423"   "9:26713423"   "9:26713423"   "9:26713402"  
 [31] "9:26713423"   "9:26713423"   "9:26713423"   "9:26713404"   "9:26713418"  
 [36] "9:26713418"   "9:26713423"   "9:26713423"   "9:26713379"   "9:26713422"  
 [41] "9:26713414"   "9:26713420"   "9:26713402"   "9:26713423"   "9:26713412"  
 [46] "9:26713423"   "9:26713423"   "9:26713423"   "9:26713423"   "9:26713420"  
 [51] "9:26713423"   "9:26713423"   "9:26713423"   "9:26713423"   "3:156523895" 
 [56] "9:26713402"   "9:26713423"   "9:26713423"   "9:26713423"   "9:26713423"  
 [61] "9:26713423"   "9:26713420"   "9:26713418"   "9:26713423"   "9:26713419"  
 [66] "17:45723492"  "9:26713422"   "9:26713423"   "9:26713423"   "9:26713420"  
 [71] "1:36463850"   "9:26713423"   "9:26713423"   "9:26713422"   "9:26713423"  
 [76] "9:26713423"   "9:26713423"   "9:26713998"   "9:26713423"   "9:26713420"  
 [81] "9:26713420"   "9:26713422"   "9:26713423"   "9:26713420"   "9:26713423"  
 [86] "9:26713423"   "9:26713423"   "9:26713423"   "9:26713423"   "9:26713420"  
 [91] "9:26713422"   "9:26713402"   "9:26713409"   "9:26713423"   "18:56852377" 
 [96] "16:35980665"  "18:32284720"  "18:57027004"  "20:372498"    "18:56852377" 
[101] "16:35980665"  "18:32284721"  "18:32284721"  "20:372498"    "20:372471"   
[106] "5:1292029"    "22:10749821"  "5:1262800"    "22:10749818"  "22:10749818" 
[111] "22:10749818"  "22:10749818"  "22:10749821"  "21:8415795"   "22:10749818" 
[116] "22:10749818"

In [58]:
represent_sites <- 
   intSiteHBV %>% dplyr::group_by(siteClusters) %>% count(siteHuman)  %>% top_n(1) %>%
   rename(representSites = siteHuman,
          representCellsInGroup = n)

Selecting by n



In [60]:
intSiteHBV <- left_join(intSiteHBV, represent_sites)

Joining, by = "siteClusters"



[1] "character"

In [61]:
readr::write_tsv(intSiteHBV, path = "./data/HBV_intSites.tsv")